# Figures
Creates figures for the report. Result data from the main spreadsheet is first loaded into "County.xlsx" and "State.xlsx" depending on the granularity of the final results. The data from these spreadsheets are used to make heat maps and other plots.

In [ ]:
#Packages
import plotly.express as px
import pandas as pd

#Get county outlines json
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    

print('Packages Loaded.')

In [ ]:

def usheatmap(df, colname, title, div=False, colorscale="OrRd",label = "",maxscale=None):
    #create us county level heatmap for column of data frame with column 'FIPS'
    if maxscale==None:
        maxscale=df[colname].max()
    #divergent heatmap
    if div:
            fig = px.choropleth(df, geojson=counties, locations='FIPS', color=colname,
                        color_continuous_scale=colorscale,
                        color_continuous_midpoint = 0,
                        scope="usa",
                        labels={colname:label}
                        )
    #continuous color range
    else:    
        fig = px.choropleth(df, geojson=counties, locations='FIPS', color=colname,
                            color_continuous_scale=colorscale,
                            range_color=(df[colname].min(),maxscale),
                            scope="usa",
                            labels={colname:label},
                            )
    fig.update_layout(title_text=title, title_x=0.5)
    fig.update_layout(width=1000, height=400, margin={"r":0,"t":25,"l":0,"b":0})#
    fig.show()
    fig.write_image(colname+'.jpeg')
    


## County maps for Health Effects and NPV

In [ ]:
# Add NPV Maps and Figures Here

fil='CountyData.xlsx'

df=pd.read_excel(fil,dtype={'FIPS':str})

usheatmap(df,'gasstoves',
          "Cost of Asthma from Gas Stoves",
           label ="$/Household/Yr", colorscale='OrRd', div=False)
usheatmap(df,'coldstress',
          "Cost of Cold Stress from Inefficient Homes",
           label ="$/Household/Yr", colorscale='OrRd', div=False, maxscale=25000)
usheatmap(df,'Annual Energy Savings',
          "Annual Energy Savings from Retrofits",
           label ="$/Household/Yr", colorscale='OrRd', div=False)
usheatmap(df,'NPV_e',
          "NPV with Energy Use Only",
           label ="$/Household", colorscale='RdBu', div=True)

usheatmap(df,'Private Health Savings',
          "Private Health Benefits",
           label ="$/Household/Yr", colorscale='OrRd', div=False, maxscale=180)
usheatmap(df,'NPV_priv',
          "NPV with Private Health Benefits and Energy Use",
           label ="$/Household", colorscale='RdBu', div=True)
usheatmap(df,'Total Annual Health Savings',
          "Public and Private Annual Health Benefits from Retrofits",
           label ="$/Household/Yr", colorscale='OrRd', div=False, maxscale=35000)
usheatmap(df,'NPV_health',
          "NPV with Public and Private Benefits",
           label ="$/Household", colorscale='Blues', div=False, maxscale=800000)
usheatmap(df,'NPV_climate',
          "NPV with Public and Private Benefits including Social Cost of Carbon ($/Household)",
           label ="$/Household", colorscale='Blues', div=False, maxscale=850000)


## NPV Violin/ Box Plots

In [ ]:
# Violin plots
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

#Violin Plots of Metrics
fig, ax = plt.subplots()
# build a violin plot
ax.violinplot(np.column_stack((df['NPV_e'], df['NPV_priv'], df['NPV_health'])), vert=False, showmeans=True, showmedians=True)

# add title and axis labels
ax.set_title('NPV')
# ax.set_xlabel('')
ax.set_xlabel('NPV ($)')

# add x-tick labels
yticklabels = ['Energy Only', 'Private', 'Public+Private']
ax.set_yticks([1,2,3])
ax.set_yticklabels(yticklabels)
f = lambda x, pos: f'${x/10**6:,.2f}M'
ax.xaxis.set_major_formatter(FuncFormatter(f))
# add horizontal grid lines
ax.xaxis.grid(True)

# show the plot
plt.show()


# Box Plot
fig, ax = plt.subplots()
# build a violin plot
ax.boxplot(np.column_stack((df['NPV_e'], df['NPV_priv'], df['NPV_health'])), vert=False)

# add x-tick labels
yticklabels = ['Energy Only', 'Private', 'Public+Private']
ax.set_yticks([1,2,3])
ax.set_yticklabels(yticklabels)
f = lambda x, pos: f'${x/10**6:,.2f}M'
ax.xaxis.set_major_formatter(FuncFormatter(f))

# add horizontal grid lines
plt.axvline(0, color='lightgrey')
fig.set_figwidth(10)
fig.set_figheight(4)
# show the plot
plt.show()


## Energy Burden Plots

In [ ]:
df['Burden_pre_deltat'] = df['Burden_pre_tax'] - df['Burden_pre_nt']
df['Burden_delta_nt'] =  df['Burden_post_nt'] -df['Burden_pre_nt']
df['Burden_delta_delta'] = df['Burden_post_tax']- df['Burden_pre_nt']

usheatmap(df,'Burden_pre_nt',
          "Baseline Energy Burden",
           label ="%", colorscale='Greys', div=False)
usheatmap(df,'Burden_pre_deltat',
          "Change in Energy Burden with a $51/MT Carbon Price",
           label ="%", colorscale='OrRd', div=False)
usheatmap(df,'Burden_delta_nt',
          "Change in Energy Burden with Retrofits",
           label ="%", colorscale='Blues_r', div=False)
usheatmap(df,'Burden_delta_delta',
          "Change in Energy Burden with Retrofits + Carbon Price",
           label ="%", colorscale='RdBu_r', div=True)


# usheatmap(df,'Burden_pre_tax',
#           "Existing Energy Burden (Expenditures/Income) with 51$/ton Carbon Tax",
#            label ="%", colorscale='OrRd', div=False)
# usheatmap(df,'Burden_post_nt',
#           "Energy Burden (Expenditures/Income) with Retrofits",
#            label ="%", colorscale='OrRd', div=False)
# usheatmap(df,'Burden_post_tax',
#           "Energy Burden (Expenditures/Income) with Retrofits and 51$/ton Carbon Tax",
#            label ="%", colorscale='OrRd', div=False)

## Energy Burden Violin Plots

In [ ]:
# Violin plots
import matplotlib.pyplot as plt
import numpy as np

#Violin Plots of Metrics
fig, ax = plt.subplots()
# build a violin plot
violin_parts = ax.violinplot(np.column_stack((df['Burden_pre_nt'], df['Burden_pre_tax'], df['Burden_post_nt'], df['Burden_post_tax'])), 
                             vert=False, showmeans=True, showmedians=False)

for pc in violin_parts['bodies']:
    pc.set_facecolor('orange')
    pc.set_edgecolor('orange')
violin_parts['cmeans'].set_color('darkorange')
# violin_parts['cmedians'].set_color('red')
violin_parts['cbars'].set_color('darkorange')
violin_parts['cmins'].set_color('darkorange')
violin_parts['cmaxes'].set_color('darkorange')

# add title and axis labels
ax.set_title('Energy Burden Ranges in Different Scenarios')
# ax.set_xlabel('')
ax.set_xlabel('Energy Expenditures/Income')

# add x-tick labels
yticklabels = ['Baseline', 'Baseline with Tax', 'Retrofits', 'Retrofits and Tax']
ax.set_yticks([1,2,3,4])
ax.set_yticklabels(yticklabels)

# add horizontal grid lines
ax.xaxis.grid(True)

# show the plot
plt.show()


## State Health Effect Maps

In [ ]:
#load data
fil='CountyData.xlsx'
fil2='StateData.xlsx'

df=pd.read_excel(fil,dtype={'FIPS':str})
# print(df)
df2=pd.read_excel(fil2)
# print(df2)

#name is the title of the data frame
#Use for Titling Health Figures
fig_title_dict = {'gasstoves': 'Cost of Asthma from Gas Stoves ($/Household/Year)', 
                  'weatherization': 'Cost of Asthma from Poor Insulation and Lack of Air Sealing', 
                  'heatstress': 'Cost of Heat Stress from Inefficient Housing', 
                  'coldstress': 'Cost of Cold Stress ($/Household/Year)',
                  'workdays': 'Cost of Missed Workdays',
                  'GHG':'Average Annual Residential GHG Emissions for Households Below 200FPL (kg CO2e/Household/Year)',
                  'hot':'Cost of Morbidity and Mortality from Heat Stress ($/Household/Yr)','cold':''}


In [ ]:
# create state level figure
#'Code' should be two letter state code

def usheatmap_state(name):
    
    fig = px.choropleth(df2,
                    locations='State Code',
                    color=name,
                    color_continuous_scale='OrRd',
                    #hover_name='State',
                    locationmode='USA-states',
                    labels={name:'$/Household/Yr'},
                    scope='usa')
    fig.update_layout(title_text=fig_title_dict[name], title_x=0.5)
    fig.update_layout(width=1000, height=400, margin={"r":0,"t":25,"l":0,"b":0})#
    fig.show()
    fig.write_image(name+'.jpeg')
    
# usheatmap_state('weatherization')

In [ ]:

usheatmap_state("hot")
usheatmap_state("workdays")
usheatmap_state("weatherization")